# Metadata

* Title: **Final Project Report**
* Class: DS 5100
* Date: 12/05/2022
* Student Name: Sirish Desai
* Student Net ID: skd3nz
* This URL: https://github.com/SirishKDesai/DS5100-2022-08-SirishKDesai/blob/main/Final%20Project/skd3nzMontecarloSimulator/final-project-submission.ipynb
* GitHub Repo URL: https://github.com/SirishKDesai/DS5100-2022-08-SirishKDesai/tree/main/Final%20Project/skd3nzMontecarloSimulator

# The Monte Carlo Module

In [1]:
import pandas as pd
import numpy as np

class DieGame():
    """
    This Class is used in combination of the Game Class and Analyzer Class to 
    create a Monte Carlo Simulator. The die has multiple sides and weights that 
    can be "rolled" to select a face.
    """
    def __init__(self, faces):
        """
        This function requires a set of already instiated faces as a list.
        The Class creates a private Dataframe to store the Faces. 
        The weights of the faces are initialized as 1s, unless changed by using the change weight function.

        Parameters
        ------
        faces: list
            has to be an array of faces
            examples: [1,2,3,4]
        """
        self.weights = np.ones(len(faces))
        self.faces = faces
        self._FacesWeights = pd.DataFrame({
            'Face': self.faces,
            'Weights' : self.weights
        })

    
    def change_weight(self,face, newweight):
        """
        This function changes the weight of the faces of the "die" that you use.
        The function checks if the Face is in the Dataset, and then will change the weight to whatever the second input was.

        Parameters
        ------

        face: any
            have to be a number in the array of faces passed to the __init__ function
            examples: 2
        newweight: int
            changes the weight of the face that was passed first
            examples: 3
        """
        faceplace = self._FacesWeights['Face'].to_numpy()
        if face not in faceplace:
            return ValueError("Not a Valid Face")
        indexed = list(self._FacesWeights.index[self._FacesWeights['Face'] ==face])[0]
        self._FacesWeights.loc[indexed,'Weights']= newweight
        

    def roll_die(self, rolls = 1):
        """
        This function randomly picks a face from the array of faces. Also takes into account the weights if Change Weight was used.
        Will roll once if no other number is put in.

        Parameters
        ------
        rolls = 1: int
            requires any integer >1.
            example: 100
        """
        rolled = self._FacesWeights.sample(n= rolls, replace = True, weights = 'Weights').reset_index(drop=True)
        return list(rolled['Face'])

    def show(self):
        """
        This function shows the Dataframe of Faces and Weights.
        Does not require a parameter.
        """
        return (self._FacesWeights)    



class Games():
    """
    This Class is used in combination of the Die Class and Analyzer Class. 
    This class rolls the instiated dice of the same kind one or more times.
    This class has the function play and show.
    """
    def __init__(self, dieobjects):
        """
        This function requires a list of one of more similarly defined dice.
        The Class uses the Class DieGame to create the list of die and change their weights.
    
        Parameters
        ------
        dieobject: list
            example:    die3 = {1:1,2:1,3:1,4:1}
                        dice = {1:die1}
                        for iterable,theDie in dice.items():
                            dice[iterable] = DieGame(theDie.keys())
                            for key in theDie.keys():
                                dice[iterable].change_weight(key, theDie[key])
                        dice.values() #pass into the class
                        game = Games(dice.values())
        """
        self.dieobjects = list(dieobjects)
        
    def play(self, rollnumber=1):
        """
        This function uses the die that was instaniated by the __init__ function 
        to randomly choose a number of the faces. The number chosen will also vary 
        by the weight that was given along with the die.
        The function will roll the die once unless another integer number is used.
        The rolled number will stored in a private dataframe called _rolled.

        Parameters
        ------
        rollnumber =1 :int
            example: 10000
                    game = Games(dice.values())
                    game.play(10000)
        """
        RolledNumber = []
        for dice in self.dieobjects:
            RolledNumber.append(dice.roll_die(rollnumber))
        RolledNumber = np.array([np.array(x) for x in RolledNumber]).T
        self._rolled = pd.DataFrame(RolledNumber, columns= range(1,len(self.dieobjects)+1), index = range(1,rollnumber+1))
        

    def show(self, NorW = 'wide'):
        """
        This function takes the rolled dataframe that was made and returns it to the user.
        The function can be ran by itself, which would return an unstacked (each role is 
        an observation and each column is a feature and each cell shows the resulting
        face for the die on the roll) dataframe, but if given the parameter 'narrow' it will 
        stack (two-column index with the roll number and the die number and a single column for the face rolled) 
        the dataframe. If given a different parameter the function will return 'ValueError Invalid Option.'

        Parameters
        ------
        NorW: 'narrow' | 'wide'
            example: game.show() | game.show('narrow')
        """
        if NorW == 'narrow':
            return (self._rolled.stack())
        if NorW == 'wide': 
            return self._rolled
        if NorW != 'wide' or 'narrow':
            return ValueError("Invalid Option")



class Analyzer():
    """
    This Class is used in combination of the Die Class and Game Class. 
    This class specifically takes the reults of a single game and computes various descriptive
    statistical properties about it.
    This class has the function faceperroll, combo, and jackpot.
    """
    def __init__(self,objects):
        """
        This function requires a game object as its input parameter.
        It analyzes the numbers that were rolled and puts them into a data frame. It then
        allows the user to see the amount of of times a given face is rolled in each event.
        Then from the other functions allows the user to see the distinct combinations of faces rolled, 
        along with their counts and to see how many times the game resulted in all faces 
        being identical.

        Parameters
        ------
        objects: list
            example:    die3 = {1:1,2:1,3:1,4:1}
                dice = {1:die1}
                for iterable,theDie in dice.items():
                    dice[iterable] = DieGame(theDie.keys())
                    for key in theDie.keys():
                        dice[iterable].change_weight(key, theDie[key])
                dice.values() #pass into the class
                game = Games(dice.values())
                game.play(10)
                game.show()
                Analyzed = Analyzer(game)
        """
        self.objects = objects
        self._faces = list(self.objects.dieobjects[0].faces)


    def faceperroll(self):
        """
        This function requires no inputs and does not return anything for the user to see.
        This function stores the results of how many times a given face is rolled in each 
        event in the game function. The dataframe is a public attribute and has an index of the 
        roll number and face values as columns.

        Parameters
        ------
        none 
        """
        self.data = pd.DataFrame(columns = self._faces, index = range(1,len(self.objects.show())+1))
        for index,series in self.objects.show().iterrows():
            events = []
            event = (series.value_counts().to_dict())
            for ithappened in self._faces: 
                if ithappened in event:
                    events.append(event[ithappened])
                else:
                    events.append(0)
            self.data.loc[index]=events 

            
    def combo(self):
        """
        This function requires no inputs and does not return anything for the user to see.
        This function computes the distinct combinations of faces rolled, along with their counts.
        The combinations are sorted and saved as a multi-columned index. The data is 
        stored in a public dataframe.

        Parameters
        ------
        none 
        """
        data_copy = self.data.copy()
        data_copy['count']= 0
        self.something = data_copy.groupby(by= self._faces).count()
        

    def jackpot(self):
        """
        This function requires no inputs, but returns an integer number for how many times
        the game results in all faces being identical. The data is stored in a public dataframe.
        The dataframe has the roll number as its named index.

        Parameters
        ------
        none 
        """
        jackpot = self.data == len(self.objects.dieobjects)
        self.jackpotresults = self.data[jackpot.any(axis=1)]
        return(jackpot.any(axis = 1).sum())
        


if __name__ == '__main__':
    die1 = {1:1,2:1,3:1,4:1}
    die2 = {1:1,2:1,3:1,4:2}
    die3 = {1:1,2:1,3:1,4:10}

    dice = {1:die1, 2:die2, 3:die3}

    for iterable,theDie in dice.items():
        dice[iterable] = DieGame(theDie.keys())
        for key in theDie.keys():
            dice[iterable].change_weight(key, theDie[key])
    game = Games(dice.values())
    game.play(10)
    game.show()
    Analyze = Analyzer(game)
    Analyze.faceperroll()
    Analyze.jackpot()

# Test Module

In [2]:
import pandas as pd
import numpy as np
import unittest
from MonteCarlo.montecarlo import Games
from MonteCarlo.montecarlo import DieGame
from MonteCarlo.montecarlo import Analyzer


class MonteCarloTests(unittest.TestCase):

    
    def test_DieInit(self):
        test_object = DieGame([1,2,3,4])
        x = test_object._FacesWeights['Face']
        self.assertEqual([1, 2, 3, 4], list(x))
    
    def test_DieChangeWeight(self):
        test_object1 = DieGame([1,2,3,4])
        test_object1.change_weight(2,3)
        x = test_object1._FacesWeights['Weights']
        self.assertTrue(3 == list(x)[1])
    
    def test_DieRollDie(self):
        test_object2 = DieGame([1,2,3,4])
        y = test_object2.roll_die()
        self.assertTrue(y[0] in [1,2,3,4])
    
    def test_DieShow(self):
        test_object3 = DieGame([1,2,3,4])
        x = test_object3._FacesWeights['Face']
        y = test_object3._FacesWeights['Weights']
        self.assertEqual(list(x), list(test_object3.show()['Face']))
        self.assertEqual(list(y), list(test_object3.show()['Weights']))

    def test_GamesInit(self):
        test_game = Games([1,2,3,4])
        x = test_game.dieobjects
        self.assertTrue(x == [1,2,3,4])

    def test_GamesPlay(self):
        die1 = {1:1,2:1,3:1,4:1}
        dice = {1:die1}
        for k,v in dice.items():
            dice[k] = DieGame(v.keys())
            for key in v.keys():
                dice[k].change_weight(key, v[key])
        test_game1 = Games(dice.values())
        test_game1.play(1)
        x = test_game1._rolled
        self.assertTrue(list(x)[0] in [1,2,3,4])

    def test_GameShow(self):
        die1 = {1:1,2:1,3:1,4:1}
        dice = {1:die1}
        for k,v in dice.items():
            dice[k] = DieGame(v.keys())
            for key in v.keys():
                dice[k].change_weight(key, v[key])
        test_game2 = Games(dice.values())
        test_game2.play(1)
        self.assertTrue(list(test_game2._rolled) == list(test_game2.show('wide')))
        self.assertFalse((test_game2._rolled.shape) == (test_game2.show('narrow').shape))
        self.assertFalse(test_game2.show('anythingelse') == list(test_game2._rolled))

    def test_AnalyzerInit(self):
        die1 = {1:1,2:1,3:1,4:1}
        dice = {1:die1}
        for k,v in dice.items():
            dice[k] = DieGame(v.keys())
            for key in v.keys():
                dice[k].change_weight(key, v[key])
        test_analyzer = Games(dice.values())
        test_analyzer.play(1)
        analyzer = Analyzer(test_analyzer)
        self.assertTrue([1,2,3,4] == analyzer._faces)
        self.assertTrue(analyzer.objects.show().values in [1,2,3,4])

    def test_AnalyzerFaceperRoll(self):
        die1 = {1:1,2:1,3:1,4:1}
        dice = {1:die1}
        for iterable,theDie in dice.items():
            dice[iterable] = DieGame(theDie.keys())
            for key in theDie.keys():
                dice[iterable].change_weight(key, theDie[key])
        test_analyzer = Games(dice.values())
        test_analyzer.play(1)
        analyzer1 = Analyzer(test_analyzer)    
        analyzer1.faceperroll()
        y = list(analyzer1.data)
        for things in y:
            self.assertTrue(things in die1.keys())

    def test_AnalyzerCombo(self):
        die1 = {1:1,2:1,3:1,4:1}
        dice = {1:die1}
        for iterable,theDie in dice.items():
            dice[iterable] = DieGame(theDie.keys())
            for key in theDie.keys():
                dice[iterable].change_weight(key, theDie[key])
        test_analyzer = Games(dice.values())
        rollnumber = 10
        test_analyzer.play(rollnumber)
        analyzer2 = Analyzer(test_analyzer)    
        analyzer2.faceperroll()
        analyzer2.combo()
        self.assertTrue(sum(analyzer2.something['count']) == rollnumber)
    
    def test_AnalyzerJackpot(self):
        die1 = {1:1,2:1,3:1,4:1}
        dice = {1:die1}
        for iterable,theDie in dice.items():
            dice[iterable] = DieGame(theDie.keys())
            for key in theDie.keys():
                dice[iterable].change_weight(key, theDie[key])
        test_analyzer = Games(dice.values())
        rollnumber = 10
        test_analyzer.play(rollnumber)
        analyzer3 = Analyzer(test_analyzer)    
        analyzer3.faceperroll()
        analyzer3.combo()
        analyzer3.jackpot()
        self.assertTrue(analyzer3.jackpot() == rollnumber)
        
    
if __name__ == '__main__':
    
    unittest.main(verbosity=2)

usage: ipykernel_launcher.py [-h] [-v] [-q] [--locals] [-f] [-c] [-b]
                             [-k TESTNAMEPATTERNS]
                             [tests ...]
ipykernel_launcher.py: error: argument -f/--failfast: ignored explicit argument 'c:\\Users\\Sirish\\AppData\\Roaming\\jupyter\\runtime\\kernel-v2-12540Yy0tFilxG5z8.json'


AssertionError: 

# Test Results

In [ ]:
test_AnalyzerCombo (__main__.MonteCarloTests) ... ok
test_AnalyzerFaceperRoll (__main__.MonteCarloTests) ... ok
test_AnalyzerInit (__main__.MonteCarloTests) ... ok
test_AnalyzerJackpot (__main__.MonteCarloTests) ... ok
test_DieChangeWeight (__main__.MonteCarloTests) ... ok
test_DieInit (__main__.MonteCarloTests) ... ok
test_DieRollDie (__main__.MonteCarloTests) ... ok
test_DieShow (__main__.MonteCarloTests) ... ok
test_GameShow (__main__.MonteCarloTests) ... ok
test_GamesInit (__main__.MonteCarloTests) ... ok
test_GamesPlay (__main__.MonteCarloTests) ... ok

----------------------------------------------------------------------
Ran 11 tests in 0.034s

OK


# Scenarios

Code blocks with your scenarios and their outputs. 

These should have appropriate import statements even though the code is now in the same notebook as the classes it calls. 

## Scenario 1

In [ ]:
import pandas as pd
import numpy as np
import unittest
from MonteCarlo.montecarlo import Games
from MonteCarlo.montecarlo import DieGame
from MonteCarlo.montecarlo import Analyzer
#Scenario 1
#1 = H
#2 = T
die1 = {1:1,2:1}

faircoins = {1:die1, 2:die1, 3:die1}

for iterable,theDie in faircoins.items():
    faircoins[iterable] = DieGame(theDie.keys())
    for key in theDie.keys():
        faircoins[iterable].change_weight(key, theDie[key])
FairGame = Games(faircoins.values())
FairGame.play(10000)
FairGame.show()
AnalyzeFairGame = Analyzer(FairGame)
AnalyzeFairGame.faceperroll()
AnalyzeFairGame.combo()
FairGame_Jackpot = AnalyzeFairGame.jackpot()

die2 = {1:5,2:1}
unfaircoins = {1:die1, 2:die2, 3:die2}
for iterable,theDie in unfaircoins.items():
    unfaircoins[iterable] = DieGame(theDie.keys())
    for key in theDie.keys():
        unfaircoins[iterable].change_weight(key, theDie[key])
UnFairGame = Games(unfaircoins.values())
UnFairGame.play(10000)
UnFairGame.show()
AnalyzeUnFairGame = Analyzer(UnFairGame)
AnalyzeUnFairGame.faceperroll()
AnalyzeUnFairGame.combo()
UnFairGame_Jackpot = AnalyzeUnFairGame.jackpot()

gamedata = pd.DataFrame({'lab':['FairGame', 'UnFairGame'], 'val':[FairGame_Jackpot, UnFairGame_Jackpot]})
ax = gamedata.plot.bar(x='lab', y='val', rot = 0)

## Scenario 2

In [ ]:
#Scenario 2
die1 = {1:1,2:1, 3:1, 4:1, 5:1, 6:1}
die2 = {1:1,2:1, 3:1, 4:1, 5:1, 6:5}
die3 = {1:5,2:1, 3:1, 4:1, 5:1, 6:1}

fairdice = {1:die1, 2:die1, 3:die1, 4:die1, 5:die1}
unfairdice = {1:die2, 2:die2, 3:die3, 4:die1, 5:die1}

for iterable,theDie in fairdice.items():
    fairdice[iterable] = DieGame(theDie.keys())
    for key in theDie.keys():
        fairdice[iterable].change_weight(key, theDie[key])
FairGame = Games(fairdice.values())
FairGame.play(10000)
FairGame.show()
AnalyzeFairGame = Analyzer(FairGame)
AnalyzeFairGame.faceperroll()
AnalyzeFairGame.combo()
FairGame_Jackpot = AnalyzeFairGame.jackpot()

for iterable,theDie in unfairdice.items():
    unfairdice[iterable] = DieGame(theDie.keys())
    for key in theDie.keys():
        unfairdice[iterable].change_weight(key, theDie[key])
UnFairGame = Games(unfairdice.values())
UnFairGame.play(10000)
UnFairGame.show()
AnalyzeUnFairGame = Analyzer(UnFairGame)
AnalyzeUnFairGame.faceperroll()
AnalyzeUnFairGame.combo()
UnFairGame_Jackpot = AnalyzeUnFairGame.jackpot()

gamedata = pd.DataFrame({'lab':['FairGame', 'UnFairGame'], 'val':[FairGame_Jackpot, UnFairGame_Jackpot]})
ax = gamedata.plot.bar(x='lab', y='val', rot = 0)

SortedFairGame = AnalyzeFairGame.something['count'].sort_values(ascending = False)
SortedFairGameIndex = AnalyzeFairGame.something['count'].sort_values(ascending = False).index
count = 0
FairGameValues = []
FairGameIndexes = []

for Top10 in SortedFairGame:
    FairGameValues.append(Top10)
    count +=  1
    if count == 10:
        break
counts = 0
for Top10Indexes in SortedFairGameIndex:
    FairGameIndexes.append(Top10Indexes)
    counts +=1
    if counts == 10:
        break

gamedata = pd.DataFrame({'FairGameIndexes':FairGameIndexes, 'FairGameCounts':FairGameValues})
ax = gamedata.plot.bar(x='FairGameIndexes', y='FairGameCounts')

SortedUnFairGame = AnalyzeUnFairGame.something['count'].sort_values(ascending = False)
SortedUnFairGameIndex = AnalyzeUnFairGame.something['count'].sort_values(ascending = False).index
count = 0
UnFairGameValues = []
UnFairGameIndexes = []

for Top10 in SortedUnFairGame:
    UnFairGameValues.append(Top10)
    count +=  1
    if count == 10:
        break
counts = 0
for Top10Indexes in SortedUnFairGameIndex:
    UnFairGameIndexes.append(Top10Indexes)
    counts +=1
    if counts == 10:
        break

gamedata = pd.DataFrame({'UnFairGameIndexes':UnFairGameIndexes, 'UnFairGameCounts':UnFairGameValues})
ax = gamedata.plot.bar(x='UnFairGameIndexes', y='UnFairGameCounts')


## Scenario 3

In [ ]:
import pandas as pd
import numpy as np
import unittest
from MonteCarlo.montecarlo import Games
from MonteCarlo.montecarlo import DieGame
from MonteCarlo.montecarlo import Analyzer
Alphabet= {
'A':	8.4966,
'B':	2.0720,
'C':	4.5388,
'D':	3.3844,
'E':	11.1607,
'F':	1.8121,
'G':	2.4705,
'H':	3.0034,
'I':	7.5448,
'J':	0.1965,
'K':	1.1016,
'L':	5.4893,
'M':	3.0129,
'N':    6.6544,
'O':	7.1635,
'P':	3.1671,
'Q':	0.1962,
'R':	7.5809,
'S':	5.7351,
'T':	6.9509,
'U':	3.6308,
'V':	1.0074,
'W':	1.2899,
'X':	0.2902,
'Y':	1.7779,
'Z':	0.2722}

AlphabetCoins = {1:Alphabet, 2:Alphabet, 3:Alphabet, 4:Alphabet, 5:Alphabet}

for iterable,theAlphabet in AlphabetCoins.items():
    AlphabetCoins[iterable] = DieGame(theAlphabet.keys())
    for key in theAlphabet.keys():
        AlphabetCoins[iterable].change_weight(key, theAlphabet[key])
AlphabetGame = Games(AlphabetCoins.values())
AlphabetGame.play(1000)
AlphabetGame.show()

for i in range(10):
    print(AlphabetGame.show().sample(n=10))
#I saw about 26 times I saw a word that looks like an English word in all the samples so 26/100, so 26% of English words are in the data


# Directory Listing

A code block that executes the following bash command: 

```bash
!ls -lRF -o
```

In [ ]:
$ !ls -lRF -o
ls -lr -lRF -o
.:
total 4
drwxr-xr-x 1 Sirish 0 Dec  5 14:26 skd3nzMontecarloSimulator/

./skd3nzMontecarloSimulator:
total 69
-rw-r--r-- 1 Sirish   385 Dec  2 17:12  setup.py
-rw-r--r-- 1 Sirish  7115 Dec  2 17:46  README.md
drwxr-xr-x 1 Sirish     0 Dec  5 14:26  MontecarloTests/
-rw-r--r-- 1 Sirish 48590 Dec  5 15:10  MonteCarlo_demo.ipynb
drwxr-xr-x 1 Sirish     0 Dec  2 16:54  MonteCarlo/
-rw-r--r-- 1 Sirish  1061 Dec  2 17:48 'License File.md'

./skd3nzMontecarloSimulator/MontecarloTests:
total 13
-rw-r--r-- 1 Sirish  679 Dec  5 14:26 montecarlo_tests_results.txt
-rw-r--r-- 1 Sirish 4576 Dec  5 14:18 montecarlo_tests.py
-rw-r--r-- 1 Sirish   59 Dec  5 14:15 __init__.py

./skd3nzMontecarloSimulator/MonteCarlo:
total 17
-rw-r--r-- 1 Sirish 9626 Dec  5 14:07 montecarlo.py
drwxr-xr-x 1 Sirish    0 Dec  5 14:36 __pycache__/
-rw-r--r-- 1 Sirish  126 Dec  5 14:15 __init__.py

./skd3nzMontecarloSimulator/MonteCarlo/__pycache__:
total 13
-rw-r--r-- 1 Sirish 10056 Dec  5 14:36 montecarlo.cpython-39.pyc
-rw-r--r-- 1 Sirish   303 Dec  5 14:36 __init__.cpython-39.pyc


# Installation Output Listing
    
A code block that executes the code to install your your package and outputs a successful installation.

In [ ]:
pip install.

In [ ]:
Processing c:\users\sirish\desktop\class\ds 5100\final project\skd3nzmontecarlosimulator
Requirement already satisfied: numpy>=1.11.1 in c:\users\sirish\anaconda3\lib\site-packages (from Monte-Carlo-Simulator==1.0.0) (1.21.5)
Requirement already satisfied: matplotlib>=1.5.1 in c:\users\sirish\anaconda3\lib\site-packages (from Monte-Carlo-Simulator==1.0.0) (3.5.1)
Requirement already satisfied: packaging>=20.0 in c:\users\sirish\anaconda3\lib\site-packages (from matplotlib>=1.5.1->Monte-Carlo-Simulator==1.0.0) (21.3)
Requirement already satisfied: cycler>=0.10 in c:\users\sirish\anaconda3\lib\site-packages (from matplotlib>=1.5.1->Monte-Carlo-Simulator==1.0.0) (0.11.0)
Requirement already satisfied: pillow>=6.2.0 in c:\users\sirish\anaconda3\lib\site-packages (from matplotlib>=1.5.1->Monte-Carlo-Simulator==1.0.0) (9.0.1)
Requirement already satisfied: kiwisolver>=1.0.1 in c:\users\sirish\anaconda3\lib\site-packages (from matplotlib>=1.5.1->Monte-Carlo-Simulator==1.0.0) (1.3.2)
Requirement already satisfied: python-dateutil>=2.7 in c:\users\sirish\anaconda3\lib\site-packages (from matplotlib>=1.5.1->Monte-Carlo-Simulator==1.0.0) (2.8.2)
Requirement already satisfied: pyparsing>=2.2.1 in c:\users\sirish\anaconda3\lib\site-packages (from matplotlib>=1.5.1->Monte-Carlo-Simulator==1.0.0) (3.0.4)
Requirement already satisfied: fonttools>=4.22.0 in c:\users\sirish\anaconda3\lib\site-packages (from matplotlib>=1.5.1->Monte-Carlo-Simulator==1.0.0) (4.25.0)
Requirement already satisfied: six>=1.5 in c:\users\sirish\anaconda3\lib\site-packages (from python-dateutil>=2.7->matplotlib>=1.5.1->Monte-Carlo-Simulator==1.0.0) (1.16.0)
Building wheels for collected packages: Monte-Carlo-Simulator
  Building wheel for Monte-Carlo-Simulator (setup.py): started
  Building wheel for Monte-Carlo-Simulator (setup.py): finished with status 'done'
  Created wheel for Monte-Carlo-Simulator: filename=Monte_Carlo_Simulator-1.0.0-py3-none-any.whl size=6428 sha256=0b035952424dc5eab287ce51e76a6ac2dcdda9c0ac0980fc34e9f72f6930f9de
  Stored in directory: C:\Users\Sirish\AppData\Local\Temp\pip-ephem-wheel-cache-hfgazr4f\wheels\99\ca\05\56a6da866bb4a330f04f1e769cd3f6b34575a452bf3eb4b423
Successfully built Monte-Carlo-Simulator
Installing collected packages: Monte-Carlo-Simulator
Successfully installed Monte-Carlo-Simulator-1.0.0
Note: you may need to restart the kernel to use updated packages.
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.